# 기계학습 팀 소믈리에 와인 등급 예측
## kNN 0521
### KNeighborsRegressor 시도

#### 분류기 말고 회귀도 있다는 것을 찾아서 시도해본 것

In [13]:
import numpy as np
import pandas as pd

In [14]:
# csv를 가져와 넘파이배열 형태의 데이터셋으로 가공하는 과정
import csv
f1=open('winequality-red.csv','r',encoding='UTF-8')
f2=open('winequality-white.csv','r',encoding='UTF-8')
red_wine_data = csv.reader(f1)
white_wine_data = csv.reader(f2)
raw1=[]# 가공 전 레드와인 데이터를 넣어둘 리스트
raw2=[]# 가공 전 화이트와인 데이터를 넣어둘 리스트

total_data=[]# 가공된 모든 와인 데이터를 넣어둘 리스트

features=[]# 특성 이름을 모아둘 리스트
feat=[]# csv에 "특성이름" 의 형태로 들어가 있어서 ""을 제거하기 전 특성 이름 리스트
target=[]# 정답인 등급을 모아둘 리스트

for r in red_wine_data:
    r=r[0].split(';')
    r.insert(0,'1')#red가 1
    raw1.append(r)

for w in white_wine_data:
    w=w[0].split(';')
    w.insert(0,'0')#white가 0
    raw2.append(w)

raw1[0].pop(0)# 와인의 색 정보로 위에서 넣어둔 1을 빼냄
feat=raw1[0]
for f in feat:
    s=f[1:len(f)-1]# ""을 제거하는 부분
    features.append(s)
features.insert(0,'color')

# csv의 첫번째 행이 특성 정보이므로 그것을 없애는 작업
raw1.pop(0)
raw2.pop(0)

for i1 in raw1:
    total_data.append(i1)

for i2 in raw2:
    total_data.append(i2)

for t in total_data:
    target.append(t.pop())


total_data=np.array(total_data)
total_data = total_data.astype(np.float64)# 실행했을때 밑의 경고문이 떠서 추가
"""
FutureWarning:
Beginning in version 0.22,
arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'.
It is recommended that you convert the array to a float dtype before using it in scikit-learn
"""

import copy

target_5=copy.deepcopy(target) # 1~4 / 5~10
target_6=copy.deepcopy(target) # 1~5 / 6~10
target_7=copy.deepcopy(target) # 1~6 / 7~10

for i in range(0,len(target_5)):
    if int(target_5[i])<5:
        target_5[i]='0'
    else:
       target_5[i]='1'
        
for i in range(0,len(target_6)):
    if int(target_6[i])<5:
        target_6[i]='0'
    else:
       target_6[i]='1'
        
for i in range(0,len(target_7)):
    if int(target_7[i])<5:
        target_7[i]='0'
    else:
       target_7[i]='1'
        
target_5=np.array(target_5)
target_5 = target_5.astype(np.float64)
target_6=np.array(target_6)
target_6 = target_6.astype(np.float64)
target_7=np.array(target_7)
target_7 = target_7.astype(np.float64)

f1.close()
f2.close()

#위에서 copy가 일어나므로 밑에서 처리
target=np.array(target)
target = target.astype(np.float64)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [16]:
# k의 차이에 따른 결과를 보기 위해, 다른 데이터셋에 따른 결과를 보기 위해 편리하게 사용할 목적으로 정의한 함수
def knr_by_n(n,x,y,test,pred):
    #매개변수는 순서대로 k, 훈련데이터 특성배열, 훈련데이터 타겟배열, 테스트셋 특성배열, 테스트셋 타겟배열
    reg=KNeighborsRegressor(n_neighbors=n, weights='distance')#k=n인 knn
    reg.fit(x, y)
    print("n = ", n)
    print("테스트 세트 예측:\n{}".format(reg.predict(test)))
    print("테스트 세트 R^2: {:.2f}".format(reg.score(test, pred)))
    return reg.score(test, pred)

### 다중분류

In [17]:
# 훈련 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(total_data, target, random_state=0)
# 훈련 데이터 크기 확인
print(x_train.shape)
print(y_train.shape)
# 테스트 데이터 크기 확인
print(x_test.shape)
print(y_test.shape)

(4872, 12)
(4872,)
(1625, 12)
(1625,)


In [18]:
l=[]# 반환된 정확도를 모아둘 배열
for i in range(1,100):#가장 가까운 1개 부터 시작해서 자신을 제외한 모든 것에 대해 kNN 시행(전부 다 하면 진짜 오래 걸립니다ㅠㅜ)
    #최대 k = 1625
    l.append(knr_by_n(i,x_train,y_train,x_test,y_test))
best=float(l[0])
for i in l:
    if i>best:
        best=i
print(best)
print("k가 " + str(l.index(best)+1) + "일때 정확도 " + str(best) + "가 최대 정확도")

n =  1
테스트 세트 예측:
[6. 6. 6. ... 5. 6. 5.]
테스트 세트 R^2: -0.02
n =  2
테스트 세트 예측:
[6. 6. 6. ... 5. 6. 5.]
테스트 세트 R^2: 0.15
n =  3
테스트 세트 예측:
[5.6691215 6.        6.        ... 5.3096962 6.        5.       ]
테스트 세트 R^2: 0.22
n =  4
테스트 세트 예측:
[5.74709314 6.         5.78606775 ... 5.47270936 6.         5.        ]
테스트 세트 R^2: 0.26
n =  5
테스트 세트 예측:
[5.60575291 6.         5.64753841 ... 5.56369726 6.         5.        ]
테스트 세트 R^2: 0.28
n =  6
테스트 세트 예측:
[5.51238125 6.         5.55051986 ... 5.627905   6.         5.        ]
테스트 세트 R^2: 0.30
n =  7
테스트 세트 예측:
[5.5768553  6.         5.47878506 ... 5.67417307 6.         5.        ]
테스트 세트 R^2: 0.31
n =  8
테스트 세트 예측:
[5.50998399 6.         5.42992632 ... 5.70959963 6.         5.        ]
테스트 세트 R^2: 0.32
n =  9
테스트 세트 예측:
[5.56063833 6.         5.39011602 ... 5.73755729 6.         5.        ]
테스트 세트 R^2: 0.33
n =  10
테스트 세트 예측:
[5.50878383 6.         5.3571723  ... 5.75937126 6.         5.        ]
테스트 세트 R^2: 0.33
n =  11
테스트 세트 예측:
[5.54949263

n =  79
테스트 세트 예측:
[5.70974937 6.         5.59378072 ... 5.80455666 6.         5.        ]
테스트 세트 R^2: 0.34
n =  80
테스트 세트 예측:
[5.72201043 6.         5.58849593 ... 5.79791694 6.         5.        ]
테스트 세트 R^2: 0.34
n =  81
테스트 세트 예측:
[5.7246193  6.         5.58335087 ... 5.78320817 6.         5.        ]
테스트 세트 R^2: 0.34
n =  82
테스트 세트 예측:
[5.73634645 6.         5.578311   ... 5.7769018  6.         5.        ]
테스트 세트 R^2: 0.34
n =  83
테스트 세트 예측:
[5.73873052 6.         5.5734199  ... 5.77868022 6.         5.        ]
테스트 세트 R^2: 0.34
n =  84
테스트 세트 예측:
[5.741061   6.         5.56861266 ... 5.78042944 6.         5.        ]
테스트 세트 R^2: 0.34
n =  85
테스트 세트 예측:
[5.74334952 6.         5.56390852 ... 5.78996818 6.         5.        ]
테스트 세트 R^2: 0.34
n =  86
테스트 세트 예측:
[5.73687345 6.         5.55929815 ... 5.79159115 6.         5.        ]
테스트 세트 R^2: 0.34
n =  87
테스트 세트 예측:
[5.730597   6.         5.55476255 ... 5.79318858 6.         5.        ]
테스트 세트 R^2: 0.34
n =  88
테스트 세트 예측:
[5.741269

### 1-4 / 5-10

In [19]:
# 훈련 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(total_data, target_5, random_state=0)
# 훈련 데이터 크기 확인
print(x_train.shape)
print(y_train.shape)
# 테스트 데이터 크기 확인
print(x_test.shape)
print(y_test.shape)

(4872, 12)
(4872,)
(1625, 12)
(1625,)


In [20]:
l=[]# 반환된 정확도를 모아둘 배열
for i in range(1,100):#가장 가까운 1개 부터 시작해서 자신을 제외한 모든 것에 대해 kNN 시행(전부 다 하면 진짜 오래 걸립니다ㅠㅜ)
    #최대 k = 1625
    l.append(knr_by_n(i,x_train,y_train,x_test,y_test))
best=float(l[0])
for i in l:
    if i>best:
        best=i
print(best)
print("k가 " + str(l.index(best)+1) + "일때 정확도 " + str(best) + "가 최대 정확도")

n =  1
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.27
n =  2
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.10
n =  3
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.01
n =  4
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.04
n =  5
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.05
n =  6
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.07
n =  7
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.07
n =  8
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.08
n =  9
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.08
n =  10
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.09
n =  11
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.09
n =  12
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  13
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  14
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  15
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.11
n =  16
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  17
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트

n =  89
테스트 세트 예측:
[0.97769339 1.         0.97800239 ... 0.95385764 1.         1.        ]
테스트 세트 R^2: 0.10
n =  90
테스트 세트 예측:
[0.9778731  1.         0.97817386 ... 0.95420245 1.         1.        ]
테스트 세트 R^2: 0.10
n =  91
테스트 세트 예측:
[0.97804991 1.         0.9783402  ... 0.95454208 1.         1.        ]
테스트 세트 R^2: 0.10
n =  92
테스트 세트 예측:
[0.97822363 1.         0.97850175 ... 0.95487628 1.         1.        ]
테스트 세트 R^2: 0.10
n =  93
테스트 세트 예측:
[0.97839302 1.         0.9786609  ... 0.95520363 1.         1.        ]
테스트 세트 R^2: 0.10
n =  94
테스트 세트 예측:
[0.97855953 1.         0.97881736 ... 0.95552274 1.         1.        ]
테스트 세트 R^2: 0.10
n =  95
테스트 세트 예측:
[0.97872301 1.         0.97897155 ... 0.94878647 1.         1.        ]
테스트 세트 R^2: 0.10
n =  96
테스트 세트 예측:
[0.97888401 1.         0.97912332 ... 0.94914468 1.         1.        ]
테스트 세트 R^2: 0.10
n =  97
테스트 세트 예측:
[0.97904131 1.         0.97927269 ... 0.94949778 1.         1.        ]
테스트 세트 R^2: 0.10
n =  98
테스트 세트 예측:
[0.979195

In [21]:
print(l)

[-0.26602564102564097, -0.10028962811107656, -0.012330372946150803, 0.03550705480050875, 0.05364326504762629, 0.06889193798752813, 0.07453687460385783, 0.07796710772806759, 0.07869352185587342, 0.09391408587693428, 0.09111563411416768, 0.09930460279079756, 0.09979477705215645, 0.10261647276190067, 0.10545460141769392, 0.10472916042669178, 0.10500956960360286, 0.10706814692013733, 0.10759037817603188, 0.10636539254008381, 0.10881288734968131, 0.10949013551432274, 0.1084982571448706, 0.10795229995852673, 0.10733914492244223, 0.10628985581079986, 0.10699811648563207, 0.10686923102745616, 0.10656180530053838, 0.10852526120125638, 0.107190153424087, 0.10697154450915314, 0.10868386401076036, 0.10742871713109846, 0.106670478338866, 0.10751506722359772, 0.10781218962759387, 0.10684693125629041, 0.10695623042166269, 0.1052919128301959, 0.10449060182488312, 0.10349893172626157, 0.10279163080888498, 0.103108859389197, 0.10225433498968806, 0.10272535111589587, 0.1034434819258202, 0.105372583887090

### 1-5 / 6-10

In [22]:
# 훈련 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(total_data, target_6, random_state=0)
# 훈련 데이터 크기 확인
print(x_train.shape)
print(y_train.shape)
# 테스트 데이터 크기 확인
print(x_test.shape)
print(y_test.shape)

(4872, 12)
(4872,)
(1625, 12)
(1625,)


In [23]:
l=[]# 반환된 정확도를 모아둘 배열
for i in range(1,100):#가장 가까운 1개 부터 시작해서 자신을 제외한 모든 것에 대해 kNN 시행(전부 다 하면 진짜 오래 걸립니다ㅠㅜ)
    #최대 k = 1625
    l.append(knr_by_n(i,x_train,y_train,x_test,y_test))
best=float(l[0])
for i in l:
    if i>best:
        best=i
print(best)
print("k가 " + str(l.index(best)+1) + "일때 정확도 " + str(best) + "가 최대 정확도")

n =  1
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.27
n =  2
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.10
n =  3
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.01
n =  4
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.04
n =  5
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.05
n =  6
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.07
n =  7
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.07
n =  8
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.08
n =  9
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.08
n =  10
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.09
n =  11
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.09
n =  12
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  13
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  14
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  15
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.11
n =  16
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  17
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트

테스트 세트 R^2: 0.10
n =  88
테스트 세트 예측:
[0.97751017 1.         0.97782743 ... 0.95350756 1.         1.        ]
테스트 세트 R^2: 0.10
n =  89
테스트 세트 예측:
[0.97769339 1.         0.97800239 ... 0.95385764 1.         1.        ]
테스트 세트 R^2: 0.10
n =  90
테스트 세트 예측:
[0.9778731  1.         0.97817386 ... 0.95420245 1.         1.        ]
테스트 세트 R^2: 0.10
n =  91
테스트 세트 예측:
[0.97804991 1.         0.9783402  ... 0.95454208 1.         1.        ]
테스트 세트 R^2: 0.10
n =  92
테스트 세트 예측:
[0.97822363 1.         0.97850175 ... 0.95487628 1.         1.        ]
테스트 세트 R^2: 0.10
n =  93
테스트 세트 예측:
[0.97839302 1.         0.9786609  ... 0.95520363 1.         1.        ]
테스트 세트 R^2: 0.10
n =  94
테스트 세트 예측:
[0.97855953 1.         0.97881736 ... 0.95552274 1.         1.        ]
테스트 세트 R^2: 0.10
n =  95
테스트 세트 예측:
[0.97872301 1.         0.97897155 ... 0.94878647 1.         1.        ]
테스트 세트 R^2: 0.10
n =  96
테스트 세트 예측:
[0.97888401 1.         0.97912332 ... 0.94914468 1.         1.        ]
테스트 세트 R^2: 0.10
n =  97
테스트

### 1-6 / 7-10

In [24]:
# 훈련 데이터와 테스트 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(total_data, target_7, random_state=0)
# 훈련 데이터 크기 확인
print(x_train.shape)
print(y_train.shape)
# 테스트 데이터 크기 확인
print(x_test.shape)
print(y_test.shape)

(4872, 12)
(4872,)
(1625, 12)
(1625,)


In [25]:
l=[]# 반환된 정확도를 모아둘 배열
for i in range(1,100):#가장 가까운 1개 부터 시작해서 자신을 제외한 모든 것에 대해 kNN 시행(전부 다 하면 진짜 오래 걸립니다ㅠㅜ)
    #최대 k = 1625
    l.append(knr_by_n(i,x_train,y_train,x_test,y_test))
best=float(l[0])
for i in l:
    if i>best:
        best=i
print(best)
print("k가 " + str(l.index(best)+1) + "일때 정확도 " + str(best) + "가 최대 정확도")

n =  1
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.27
n =  2
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.10
n =  3
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: -0.01
n =  4
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.04
n =  5
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.05
n =  6
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.07
n =  7
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.07
n =  8
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.08
n =  9
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.08
n =  10
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.09
n =  11
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.09
n =  12
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  13
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  14
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  15
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.11
n =  16
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트 세트 R^2: 0.10
n =  17
테스트 세트 예측:
[1. 1. 1. ... 1. 1. 1.]
테스트

테스트 세트 R^2: 0.10
n =  89
테스트 세트 예측:
[0.97769339 1.         0.97800239 ... 0.95385764 1.         1.        ]
테스트 세트 R^2: 0.10
n =  90
테스트 세트 예측:
[0.9778731  1.         0.97817386 ... 0.95420245 1.         1.        ]
테스트 세트 R^2: 0.10
n =  91
테스트 세트 예측:
[0.97804991 1.         0.9783402  ... 0.95454208 1.         1.        ]
테스트 세트 R^2: 0.10
n =  92
테스트 세트 예측:
[0.97822363 1.         0.97850175 ... 0.95487628 1.         1.        ]
테스트 세트 R^2: 0.10
n =  93
테스트 세트 예측:
[0.97839302 1.         0.9786609  ... 0.95520363 1.         1.        ]
테스트 세트 R^2: 0.10
n =  94
테스트 세트 예측:
[0.97855953 1.         0.97881736 ... 0.95552274 1.         1.        ]
테스트 세트 R^2: 0.10
n =  95
테스트 세트 예측:
[0.97872301 1.         0.97897155 ... 0.94878647 1.         1.        ]
테스트 세트 R^2: 0.10
n =  96
테스트 세트 예측:
[0.97888401 1.         0.97912332 ... 0.94914468 1.         1.        ]
테스트 세트 R^2: 0.10
n =  97
테스트 세트 예측:
[0.97904131 1.         0.97927269 ... 0.94949778 1.         1.        ]
테스트 세트 R^2: 0.10
n =  98
테스트

# 다중분류: 0.35

# 이진분류: 0.11